## BurgerKing

In [1]:
# !pip install pyarrow
# !pip install fastparquet

In [128]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import f1_score

from tqdm import tqdm, trange

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Убираем ограничение отображемых колонок
pd.options.display.max_columns = None

### Загружаем датасет

In [4]:
burger_df = pd.read_parquet('datasets/train_dataset_hackaton2023_train.gzip')
burger_df.shape

(12129384, 9)

In [5]:
test_burger_df = pd.read_parquet('datasets/test_dataset_hackaton2023_test.gzip')
test_burger_df.shape

(2498034, 7)

In [6]:
# Загружаем группы продуктов
menu_df = pd.read_excel('datasets/Menu_v2.xlsx')
menu_df.shape

(1049, 2)

In [7]:
# Период за который предоставлены данные
burger_df['startdatetime'].min(), burger_df['startdatetime'].max()

(Timestamp('2022-09-02 00:00:39'), Timestamp('2023-08-01 23:57:46'))

In [8]:
# Период за который предоставлены данные в тесте
test_burger_df['startdatetime'].min(), test_burger_df['startdatetime'].max()

(Timestamp('2022-09-02 00:04:00'), Timestamp('2023-08-01 23:31:03'))

In [9]:
# Проверка что в тестовых данных есть все уникальные места которые есть в обучающей выборке
set(map(tuple,test_burger_df[['format_name', 'ownareaall_sqm']].values.tolist())) - set(map(tuple,burger_df[['format_name', 'ownareaall_sqm']].values.tolist()))

set()

In [10]:
# Объединяем тест и трайн в один датасет для общей обработки
test_burger_df['buy_post'] = np.nan
test_burger_df['date_diff_post'] = np.nan
union_burger_df = pd.concat([burger_df, test_burger_df])
union_burger_df.shape

(14627418, 9)

In [11]:
# Добавляем категории продуктов
union_burger_df = union_burger_df.merge(menu_df, on='dish_name')
union_burger_df.shape

(14627418, 10)

In [12]:
# Различить одну кафе от другой вероятно можно только по уникальности пары: 'format_name', 'ownareaall_sqm'
place_df = union_burger_df[['format_name', 'ownareaall_sqm']].drop_duplicates().sort_values(by='format_name').reset_index().rename(columns={'index':'place_id'})
place_df.shape

(692, 3)

In [13]:
# Заменяем нулевые площади на схожие, но не путая ИД мест
place_df['correct_ownareaall_sqm'] = place_df['ownareaall_sqm']
place_df.loc[(place_df['format_name'] == 'Фудкорт без туалета')&(place_df['ownareaall_sqm'] == 0), 'correct_ownareaall_sqm'] = place_df[place_df['format_name'] == 'Фудкорт без туалета']['ownareaall_sqm'].median()
place_df.loc[(place_df['format_name'] == 'Отдельно стоящий с внешней зоной')&(place_df['ownareaall_sqm'] == 0), 'correct_ownareaall_sqm'] = place_df[place_df['format_name'] == 'Отдельно стоящий с внешней зоной']['ownareaall_sqm'].median()

In [14]:
# Заносим в общий датасет ид места
union_burger_df = union_burger_df.merge(place_df, on=['format_name', 'ownareaall_sqm'])
union_burger_df.shape

(14627418, 12)

In [15]:
union_burger_df.head(3)

,customer_id,date_diff_post,buy_post,group_name,revenue,startdatetime,dish_name,ownareaall_sqm,format_name,dish_cat,place_id,correct_ownareaall_sqm
0,29891,9.0,1.0,train,69.99,2022-12-05 12:03:58,Кинг Фри станд,300.0,Отдельно стоящий без внешней зоны,Фри,0,300.0
1,197671,24.0,1.0,train,1.00,2022-09-25 16:12:58,Кинг Фри станд,300.0,Отдельно стоящий без внешней зоны,Фри,0,300.0
2,199749,NaN,0.0,train,94.99,2023-04-22 13:47:35,Кинг Фри станд,300.0,Отдельно стоящий без внешней зоны,Фри,0,300.0


### Формирование фич
Будем использовать бустинги  
Одна запись это 1 клиент  
Все данные по покупкам превращаем в фичи  

In [16]:
# burger_df[burger_df['customer_id'] == 29891]

In [17]:
# Раскладываем даты по возможным начениям
union_burger_df['date'] = union_burger_df['startdatetime'].dt.date
union_burger_df['date'] = pd.to_datetime(union_burger_df['date'])
union_burger_df['year'] = union_burger_df['startdatetime'].dt.year
union_burger_df['month'] = union_burger_df['startdatetime'].dt.month
union_burger_df['day'] = union_burger_df['startdatetime'].dt.day
union_burger_df['day_of_week'] = union_burger_df['startdatetime'].dt.day_of_week
union_burger_df['day_of_year'] = union_burger_df['startdatetime'].dt.day_of_year
union_burger_df['days_in_month'] = union_burger_df['startdatetime'].dt.days_in_month
union_burger_df['quarter'] = union_burger_df['startdatetime'].dt.quarter
union_burger_df['week'] = union_burger_df['startdatetime'].dt.week
union_burger_df['hour'] = union_burger_df['startdatetime'].dt.hour

# Для группировки данных по дням определяем начальную дату и сколько прошло дней с этого момента
date_2022 = datetime.strptime("2022-01-01", "%Y-%m-%d", ).replace(tzinfo=None)
union_burger_df['day_count'] = ((union_burger_df['startdatetime'] - date_2022) / np.timedelta64(1, 'D')).astype(int)
# Для группировки данных по три часа также устанавливаем для каждой значения группу
union_burger_df['hour_count'] = ((union_burger_df['startdatetime'] - date_2022) / np.timedelta64(3, 'h')).astype(int)

C:\Users\Profit77\AppData\Local\Temp\ipykernel_1952\3234485552.py:11: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  union_burger_df['week'] = union_burger_df['startdatetime'].dt.week


In [18]:
# Определяем время дня: утро/день/вечер
def time_of_day(row):
    hour = row.hour
    if 6 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 18:
        return 'day'
    else:
        return 'evening'

union_burger_df['times_of_day'] = union_burger_df['startdatetime'].apply(time_of_day)

In [19]:
# one_hot_encoding = pd.get_dummies(burger_df['times_of_day'], prefix='time_')
# df_concat = pd.concat([df_concat, one_hot_encoding], axis=1)
# df_concat

In [20]:
# Создаем датафрейм для итоговых агрегатов по клиентам
customer_df = pd.DataFrame()

In [21]:
# pd.DataFrame(position_by_day_df).reset_index().groupby(by=['customer_id'])['dish_name'].mean()
# burger_df.groupby(by=['customer_id'])['dish_name'].mean()

In [22]:
# Сколько позиций в заданный период времени
def calc_position_by_time(df, period_column:str='day_count'):
    position_by_time = df.groupby(by=['customer_id', period_column])['dish_name'].count()
    mean_position_by_time = pd.DataFrame(position_by_time).reset_index().groupby(by=['customer_id'])['dish_name'].mean()
    mean_position_by_time = pd.DataFrame(mean_position_by_time).reset_index()
    mean_position_by_time = mean_position_by_time.rename(columns={'dish_name':f'mean_position_by_{period_column}'}).set_index('customer_id')
    return mean_position_by_time


In [23]:
# Сколько позиций в день
mean_position_by_day = calc_position_by_time(union_burger_df, period_column='day_count')
# Сколько позиций в три часа
mean_position_by_3hour = calc_position_by_time(union_burger_df, period_column='hour_count')
# Сколько позиций в час
mean_position_by_hour = calc_position_by_time(union_burger_df, period_column='hour')

customer_df = pd.concat([customer_df, mean_position_by_day, mean_position_by_3hour, mean_position_by_hour], axis=1)
customer_df.shape

(612334, 3)

In [24]:
# Формируем сгруппированый датафрейм по посещениям за каждый день по каждому клиенту
visited_days_df = union_burger_df.groupby(by=['customer_id', 'date'])['dish_name'].count()
visited_days_df = pd.DataFrame(visited_days_df).reset_index()

In [25]:
def calc_stat_visited_by_period(period:str='1D'):
    calc_visited_df = pd.DataFrame([])
    select_group_visited = visited_days_df.set_index('date').groupby('customer_id')['customer_id'].rolling(period).count().groupby(by=['customer_id'])
    # calc_visited_df[f'count_visited_by_{period}'] = select_group_visited.count()
    calc_visited_df[f'max_visited_by_{period}'] = select_group_visited.max()
    calc_visited_df[f'mean_visited_by_{period}'] = select_group_visited.mean()
    return calc_visited_df

In [26]:
%%time
union_visited_df = pd.DataFrame(visited_days_df.set_index('date').groupby('customer_id')['customer_id'].rolling('1D').count().groupby(by=['customer_id']).count().rename('count_visited'))
for period in tqdm(['2D', '3D', '5D', '7D', '14D', '21D', '30D', '50D', '60D', '100D', '150D', '200D']):
    print(f'calc period: {period}')
    visited_df = calc_stat_visited_by_period(period=period)
    union_visited_df = pd.concat([union_visited_df, visited_df], axis=1)
union_visited_df.shape

  0%|          | 0/12 [00:00<?, ?it/s]

calc period: 2D


  8%|▊         | 1/12 [00:12<02:12, 12.04s/it]

calc period: 3D


 17%|█▋        | 2/12 [00:23<01:59, 11.94s/it]

...

 92%|█████████▏| 11/12 [02:10<00:11, 11.89s/it]

calc period: 200D


100%|██████████| 12/12 [02:22<00:00, 11.90s/it]

CPU times: total: 2min 34s
Wall time: 2min 34s


(612334, 25)

In [27]:
customer_df = pd.concat([customer_df, union_visited_df], axis=1)
customer_df.shape

(612334, 28)

In [28]:
union_burger_df.shape

(14627418, 25)

In [29]:
# Временные фичи

In [30]:
# В качестве общих признаков по дате используем последние записи по сгруппированным по клиенту данным
last_row_burger_columns = ['format_name', 'year', 'month', 'day', 
                         'day_count', 'day_of_week', 'day_of_year', 
                         'days_in_month', 'quarter', 'week', 
                         'hour_count', 'hour', 'date', 'correct_ownareaall_sqm']
last_row_df = union_burger_df[['customer_id', 'startdatetime'] + last_row_burger_columns].sort_values(by='startdatetime').groupby(by=['customer_id'])[last_row_burger_columns].last()
customer_df = pd.concat([customer_df, last_row_df], axis=1)
customer_df.shape

(612334, 42)

In [31]:
# Рабочий день или выходной/праздничный
prazdniki = ['2022-11-04', '2022-12-31', '2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', 
             '2023-01-06', '2023-02-23', '2023-02-24', '2023-03-08', '2023-05-01','2023-05-08', '2023-05-09', '2023-06-12',]
# union_burger_df['is_work_day'] = np.where(( (union_burger_df['date'].isin(prazdniki)) | (union_burger_df['day_of_week'] >=5)), 0, 1)
union_burger_df['type_day'] = np.where(( (union_burger_df['date'].isin(prazdniki)) | (union_burger_df['day_of_week'] >=5)), 'holiday', 'workday')

In [34]:
# %%time
#if need also category c with no values of 'one'
# union_burger_df.groupby(['customer_id', 'date'])['times_of_day'].apply(lambda x: (x=='morning').sum()).reset_index(name='count')
# print (df11)

In [48]:
union_burger_df[['customer_id', 'date', 'times_of_day']].drop_duplicates().shape


(14627418, 3)

In [49]:
%%time
# Считаем посещения по клиентам по временам суток
visited_time_of_day_df = union_burger_df[['customer_id', 'date', 'times_of_day']].drop_duplicates().groupby(by=['customer_id']).agg(
    # Кол-во посещений всего
    count_times_of_visited = ('times_of_day', len),
    # Кол-во посещений утром
    count_times_of_morning = ('times_of_day', lambda x: x.eq('morning').sum()),
    # Кол-во посещений днем
    count_times_of_day = ('times_of_day', lambda x: x.eq('day').sum()),
    # Кол-во посещений вечером
    count_times_of_evening = ('times_of_day', lambda x: x.eq('evening').sum()),
)
visited_time_of_day_df.shape

CPU times: total: 2min 14s
Wall time: 2min 14s


(612334, 4)

In [53]:
# Считаем долю посещений по временам суток
visited_time_of_day_df['prc_morning_visited'] = visited_time_of_day_df['count_times_of_morning']/visited_time_of_day_df['count_times_of_visited']
visited_time_of_day_df['prc_day_time_visited'] = visited_time_of_day_df['count_times_of_day']/visited_time_of_day_df['count_times_of_visited']
visited_time_of_day_df['prc_evening_visited'] = visited_time_of_day_df['count_times_of_evening']/visited_time_of_day_df['count_times_of_visited']

customer_df = pd.concat([customer_df, visited_time_of_day_df], axis=1)
customer_df.shape

(612334, 49)

In [55]:
%%time
# Считаем посещения по клиентам из разряда один день одно посещение
visited_weekday_df = union_burger_df[['customer_id', 'date', 'type_day']].drop_duplicates().groupby(by=['customer_id']).agg(
    # Кол-во посещений всего
    count_visited_by_day = ('date', len),
    # Кол-во посещений в будни    
    count_visited_workday = ('type_day', lambda x: x.eq('workday').sum()),
    # Кол-во посещений в выходные
    count_visited_holiday = ('type_day', lambda x: x.eq('holiday').sum()),
)
visited_weekday_df.shape

CPU times: total: 1min 37s
Wall time: 1min 37s


(612334, 3)

In [58]:
# Считаем долю по типу будни/выходные
visited_weekday_df['prc_visited_workday'] = visited_weekday_df['count_visited_workday']/visited_weekday_df['count_visited_by_day']
visited_weekday_df['prc_visited_holiday'] = visited_weekday_df['count_visited_holiday']/visited_weekday_df['count_visited_by_day']

customer_df = pd.concat([customer_df, visited_weekday_df], axis=1)
customer_df.shape

(612334, 54)

In [59]:
%%time
# Считаем временные агрегаты по клиентам
count_dish_cat_df = union_burger_df.groupby(by=['customer_id']).agg(
    # Кол-во позиций всего
    count_revenue = ('revenue', len),
    # Кол-во позиций типа Категория Бургер
    count_dish_cat_burger = ('dish_cat', lambda x: x.eq('Бургер').sum()),
    # Кол-во позиций типа Категория Напитки
    count_dish_cat_drink = ('dish_cat', lambda x: x.eq('Напитки').sum()),
    # Кол-во позиций типа Категория Воппер
    count_dish_cat_vopper = ('dish_cat', lambda x: x.eq('Воппер').sum()),
    # Кол-во позиций типа Категория Кофе
    count_dish_cat_koffe = ('dish_cat', lambda x: x.eq('Кофе').sum()),
    # Кол-во позиций типа Категория Фри
    count_dish_cat_fri = ('dish_cat', lambda x: x.eq('Фри').sum()),
    # Кол-во позиций типа Категория наггетсы
    count_dish_cat_naggets = ('dish_cat', lambda x: x.eq('наггетсы').sum()),
    # Кол-во позиций типа Категория Десерты
    count_dish_cat_desert = ('dish_cat', lambda x: x.eq('Десерты').sum()),
    # Кол-во позиций типа Категория Ролл
    count_dish_cat_roll = ('dish_cat', lambda x: x.eq('Ролл').sum()),
    # Кол-во позиций типа Категория чипсы
    count_dish_cat_chips = ('dish_cat', lambda x: x.eq('чипсы').sum()),
    # Кол-во позиций типа Категория Прочее
    count_dish_cat_other = ('dish_cat', lambda x: x.eq('Прочее').sum()),
    # Кол-во позиций типа Категория соус
    count_dish_cat_sous = ('dish_cat', lambda x: x.eq('соус').sum()),
    # Кол-во позиций типа Категория Пиво
    count_dish_cat_beer = ('dish_cat', lambda x: x.eq('Пиво').sum()),
    # Кол-во позиций типа Категория Шейк
    count_dish_cat_sheik = ('dish_cat', lambda x: x.eq('Шейк').sum()),
    # Кол-во позиций типа Категория Креветки
    count_dish_cat_krevetki = ('dish_cat', lambda x: x.eq('Креветки').sum()),
    # Кол-во позиций типа Категория Салфетка
    count_dish_cat_paper = ('dish_cat', lambda x: x.eq('Салфетка').sum()),
    # Кол-во позиций типа Категория Салат
    count_dish_cat_salat = ('dish_cat', lambda x: x.eq('Салат').sum()),
    # Кол-во позиций типа Категория Чай
    count_dish_cat_tie = ('dish_cat', lambda x: x.eq('Чай').sum()),
)
count_dish_cat_df.shape

CPU times: total: 12min 30s
Wall time: 12min 30s


In [63]:
# Рассчитываем долю типов продуктов
count_dish_cat_df['prc_dish_cat_burger'] = count_dish_cat_df['count_dish_cat_burger']/count_dish_cat_df['count_revenue']
count_dish_cat_df['prc_dish_cat_drink'] = count_dish_cat_df['count_dish_cat_drink']/count_dish_cat_df['count_revenue']
count_dish_cat_df['prc_dish_cat_vopper'] = count_dish_cat_df['count_dish_cat_vopper']/count_dish_cat_df['count_revenue']
count_dish_cat_df['prc_dish_cat_koffe'] = count_dish_cat_df['count_dish_cat_koffe']/count_dish_cat_df['count_revenue']
count_dish_cat_df['prc_dish_cat_fri'] = count_dish_cat_df['count_dish_cat_fri']/count_dish_cat_df['count_revenue']
count_dish_cat_df['prc_dish_cat_naggets'] = count_dish_cat_df['count_dish_cat_naggets']/count_dish_cat_df['count_revenue']
count_dish_cat_df['prc_dish_cat_desert'] = count_dish_cat_df['count_dish_cat_desert']/count_dish_cat_df['count_revenue']
count_dish_cat_df['prc_dish_cat_roll'] = count_dish_cat_df['count_dish_cat_roll']/count_dish_cat_df['count_revenue']
count_dish_cat_df['prc_dish_cat_chips'] = count_dish_cat_df['count_dish_cat_chips']/count_dish_cat_df['count_revenue']
count_dish_cat_df['prc_dish_cat_other'] = count_dish_cat_df['count_dish_cat_other']/count_dish_cat_df['count_revenue']
count_dish_cat_df['prc_dish_cat_sous'] = count_dish_cat_df['count_dish_cat_sous']/count_dish_cat_df['count_revenue']
count_dish_cat_df['prc_dish_cat_beer'] = count_dish_cat_df['count_dish_cat_beer']/count_dish_cat_df['count_revenue']
count_dish_cat_df['prc_dish_cat_sheik'] = count_dish_cat_df['count_dish_cat_sheik']/count_dish_cat_df['count_revenue']
count_dish_cat_df['prc_dish_cat_krevetki'] = count_dish_cat_df['count_dish_cat_krevetki']/count_dish_cat_df['count_revenue']
count_dish_cat_df['prc_dish_cat_paper'] = count_dish_cat_df['count_dish_cat_paper']/count_dish_cat_df['count_revenue']
count_dish_cat_df['prc_dish_cat_salat'] = count_dish_cat_df['count_dish_cat_salat']/count_dish_cat_df['count_revenue']
count_dish_cat_df['prc_dish_cat_tie'] = count_dish_cat_df['count_dish_cat_tie']/count_dish_cat_df['count_revenue']

count_dish_cat_df.shape

(612334, 35)

In [64]:
customer_df = pd.concat([customer_df, count_dish_cat_df], axis=1)
customer_df.shape

(612334, 89)

In [65]:
# Определяем таргеты для клиента, просто последня запись по сгруппированным по клиенту данным
target_df = union_burger_df.groupby(by=['customer_id'])[['buy_post', 'date_diff_post']].last()
customer_df = pd.concat([customer_df, target_df], axis=1)
customer_df.shape

(612334, 91)

In [69]:
# Сколько уникальных мест посещал клиент или только в один ходит
# count_uniq_place_df = burger_df.groupby(by=['customer_id']).agg({"place_id": pd.Series.nunique}).rename(columns={'place_id': 'count_uniq_place'})
count_uniq_place_df = union_burger_df.groupby(by=['customer_id']).agg(count_uniq_place=("place_id", pd.Series.nunique))
customer_df = pd.concat([customer_df, count_uniq_place_df], axis=1)
customer_df.shape

(612334, 92)

In [70]:
# Кол-во положительных клиентов (оставшихся) по трейну
count_good_customers_df = union_burger_df.groupby(by=['place_id', 'customer_id']).agg(count_good_customers = ('buy_post', lambda x: x.eq(1).sum())).clip(upper=1).groupby(by=['place_id']).sum()
count_good_customers_df.shape

(692, 1)

In [71]:
%%time
# Считаем агрегаты для опредлеенного места
place_features_df = union_burger_df.groupby(by=['place_id']).agg(
    # Кол-во уникальных клиентов
    count_uniq_customers = ('customer_id', pd.Series.nunique),
    # Сумма чека по всем продуктам
    sum_revenue = ('revenue', np.sum),
    # Средняя сумма по каждой позиции
    mean_one_revenue = ('revenue', np.mean),
    # Медиана сумма по каждой позиции
    median_one_revenue = ('revenue', np.median),
    # Кол-во позиций всего
    count_revenue = ('revenue', len),
    # Минимальное число дней, через сколько приходят клиенты
    min_diff_post = ('date_diff_post', np.min),
    # Медианное число дней, через сколько приходят клиенты
    median_diff_post = ('date_diff_post', np.median),
    # Максимальное число дней, через сколько приходят клиенты
    max_diff_post = ('date_diff_post', np.max),
)
place_features_df.shape

(692, 8)

In [72]:
# Добавляем Кол-во положительных клиентов по месту
place_features_df = place_features_df.merge(count_good_customers_df, on='place_id')

In [73]:
# Рассчитываем доп характеристики по месту 
place_features_df['prc_good_customers'] = place_features_df['count_good_customers']/place_features_df['count_uniq_customers']
place_features_df['mean_revenue_by_client'] = place_features_df['sum_revenue']/place_features_df['count_uniq_customers']
place_features_df['mean_revenue'] = place_features_df['sum_revenue']/place_features_df['count_revenue']

In [74]:
# Определяем приоритетное кафе у клиента
client_pop_place_df = union_burger_df.groupby(by=['customer_id', 'place_id']).agg(count_visit=('buy_post', len))
client_pop_place_df = client_pop_place_df.reset_index()

# Ищем какие места клиент посещал чаще других, в случаи совпадений берем последний
client_pop_place_df = client_pop_place_df[client_pop_place_df['count_visit'] == client_pop_place_df.groupby(['customer_id'])['count_visit'].transform(max)]
client_pop_place_df = client_pop_place_df.groupby(by=['customer_id']).last()
client_pop_place_df = client_pop_place_df.rename(columns={'place_id': 'pop_place_id', 'count_visit': 'count_visit_pop_place'})
client_pop_place_df.shape

(612334, 2)

In [75]:
customer_df = customer_df.merge(client_pop_place_df, on='customer_id')
customer_df.shape

(612334, 94)

In [77]:
customer_df = customer_df.reset_index().merge(place_features_df, left_on='pop_place_id', right_on='place_id')
customer_df = customer_df.set_index('customer_id')
customer_df.shape

(612334, 106)

In [269]:
# df_grouped = tmp_df.groupby(['customer_id', 'place_id']).agg({'count_visit':'max'})
# # df_grouped = df_grouped.reset_index()
# # df_grouped = df_grouped.rename(columns={'count_visit':'count_max'})
# # tmp_df_2 = pd.merge(tmp_df, df_grouped, how='left', on=['customer_id', 'place_id'])
# # # tmp_df_2 = tmp_df[tmp_df['count_visit'] == tmp_df['count_max']]
# # tmp_df_2
# df_grouped

In [270]:
# tmp_df[tmp_df['customer_id'] == 29891].groupby(['customer_id', 'place_id'])["count_visit"].idxmax()
# # tmp_df.groupby(['customer_id', 'place_id'])["count_visit"].idxmax()

In [271]:
# tmp_df.loc[tmp_df.groupby(['customer_id', 'place_id'])["count_visit"].idxmax()]

In [272]:
# # tmp_df2 = tmp_df.sort_values('count_visit').drop_duplicates(['customer_id', 'place_id'], keep='last')
# tmp_df2 = tmp_df.sort_values('count_visit').groupby(['customer_id', 'place_id']).tail(1)

In [384]:
# burger_df.groupby(['customer_id','month']).size().reset_index(name='freq')

In [99]:
# Частота посещения людьми кафе в определенный день
agg_func_math = {
    'day': ['mean', 'median', 'min', 'max'],
    'month': ['mean', 'median', 'min', 'max']
}
#burger_df.groupby(by=['customer_id', period_column])['dish_name'].count()
burger_df.groupby(by=['customer_id']).agg(agg_func_math).head(7)

day                     month               
                  mean median min max       mean median min max
customer_id                                                    
29891        18.911765   21.5   1  31   8.794118   12.0   1  12
30477        14.721311   15.0   1  31  10.590164   11.0  10  11
31426        14.604651   14.0   1  31   5.906977    6.0   5   7
44491         9.600000   10.0   8  11   6.300000    6.0   6   7
44939        16.720000   17.0  10  25  12.000000   12.0  12  12
45006        17.368421   18.0   7  25   5.157895    5.0   5   6
45038        12.722222    8.0   8  19   3.555556    4.0   3   4

In [100]:
# Определеяем характеристики мест

In [31]:
burger_df.shape

(12129384, 22)

# Модели

In [78]:
# Вывод графика feature importance
def plot_feature_importance(importance, names, model_name="", top_n=-1, skip_columns=[]):
    """Функция вывода feature importance
        :importance - массив важности фичей, полученный от модели
        :names - массив названий фичей
        :model_name - название модели
        :top_n - кол-во выводимых фичей
        :skip_columns: какие фичи пропустить, такое может понадобиться чтобы временно убрать 
                        из отображаемых горячие фичи, и изучить менее сильные
        :return - fi_df - feature importance датафрейм
    """
    feature_importance = np.array(importance)
    feature_names = np.array(names)
    
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)
    fi_df = fi_df[~fi_df['feature_names'].isin(skip_columns)]
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    
    plt.figure(figsize=(10,8))
    sns.barplot(x=fi_df['feature_importance'][:top_n], y=fi_df['feature_names'][:top_n])
    if top_n != -1:
        plt.title(f"{model_name} FEATURE IMPORTANCE (Top: {top_n})")
    else:
        plt.title(f"{model_name} FEATURE IMPORTANCE")
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')
    return fi_df

In [79]:
# Вывод графика ROC-AUC
def plot_roc_auc(y_true, y_pred):
    fpr, tpr, _ = roc_curve(y_true=y_true, y_score=y_pred)
    roc_auc = roc_auc_score(y_true=y_true, y_score=y_pred)

    plt.figure(figsize=(10, 3))
    plt.plot(fpr, tpr, color='darkorange',
             lw=2, label='ROC curve (area = %0.4f)' % roc_auc, alpha=0.5)

    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', alpha=0.5)

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.grid(True)
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title('Receiver operating characteristic', fontsize=16)
    plt.legend(loc="lower right", fontsize=12)
    plt.show()
    return roc_auc

In [84]:
train_customers_idx = union_burger_df[union_burger_df['group_name'] == 'train']['customer_id'].unique()
test_customers_idx = union_burger_df[union_burger_df['group_name'] == 'test']['customer_id'].unique()
train_customers_idx.shape, test_customers_idx.shape

((500000,), (112334,))

In [86]:
customer_df = customer_df.reset_index()
train_data_df = customer_df[customer_df['customer_id'].isin(train_customers_idx)]
test_data_df = customer_df[customer_df['customer_id'].isin(test_customers_idx)]

train_data_df.shape, test_data_df.shape

((500000, 107), (112334, 107))

In [200]:
# customer_df.merge(union_burger_df[['group_name', 'customer_id']], on='customer_id', how='left').shape


In [201]:
save_customer_df = customer_df.copy()
save_customer_df.loc[save_customer_df['customer_id'].isin(train_customers_idx), 'group_name'] = 'train'
save_customer_df.loc[~save_customer_df['customer_id'].isin(train_customers_idx), 'group_name'] = 'test'
save_customer_df.shape

(612334, 108)

In [203]:
save_customer_df.to_parquet('save_customer_df.parquet')